## Gridradar API
https://gridradar.net/de

In [ ]:
#!/usr/bin/python3
import requests
import json
import pprint
from lib.gridradar_api import *
from lib.basic_functions import *

In [ ]:
import config

#Gridradar.net API configuration
APItoken = config.APItoken
#InfluxDB configuration
influx_host =  config.influx_host
influx_port = config.influx_port # os.environ.get('INFLUX_PORT')
influx_username = config.influx_username
influx_password = config.influx_password
influx_database = config.influx_database
location = config.location 


https://api.gridradar.net/aggr_functions?token=4W01v742EaQ9iZj11B55vjaA4RyOtsNmu6M26EH09mFxlswbOSxA2X9bxuwZOBnpaYXtXyXf+xeQYl2LnQxev35WbGQVruURZteEpe52C0FVv08wXcb9Xm5f6Dvn/L3a


##  Metrics
**KEY**                            DESCRIPTION\
**- frequency**\
PMU selection, frequency 100ms, max period per query 1h\
**- frequency-mean-1m**\
PMU selection, 1m avg., max period per query 30d\
**- frequency-mean-1s**\
PMU selection, 1s avg., max period per query 12h\
**- frequency-ucte-median-1s**\
Median of the pmu 1 second mean frequencies\
**- net-time**\
Network time deviation of the UCTE power grid\
**- phase-angle**\
PMU selection, phase angle 100ms, max period per query 1h\
**- phase-angle-first-1m**\
PMU selection, first val. of minute, max period per query 30d\
**- phase-angle-first-1s**\
PMU selection, first val. of second, max period per query 12h\


columns	
0	"function"
1	"description"
data	
0	
0	"count"
1	"number of non-null values"
1	
0	"mean"
1	"arithmetic average of values"
2	
0	"median"
1	"middle value of the sorted list of values."
3	
0	"spread"
1	"difference between the minimum and maximum values"
4	
0	"stddev"
1	"the standard deviation of values."
5	
0	"sum"
1	"sum of values"
6	
0	"difference"
1	"difference between subsequent values"
7	
0	"min"
1	"lowest value"
8	
0	"max"
1	"greatest value"
9	
0	"first"
1	"the value with the oldest timestamp"
10	
0	"last"
1	"the value with the newest timestamp"

In [ ]:
# two different loops:
# query high_resolution data with short timerange every 30min
# query low_resolution data every 6hrs



## get data from influxDB

In [ ]:
from influxdb import DataFrameClient

clientDF = DataFrameClient(influx_host,
                           influx_port,
                           influx_username,
                           influx_password,
                           influx_database)

In [ ]:
#clientDF.query('SELECT "value" FROM "gridradar"."autogen"."frequency"')
metric='frequency'
influxquerystr=(r'SELECT last("value"), time FROM "gridradar"."autogen"."'+ metric +r'"')
response = clientDF.query(influxquerystr)

if len(response) == 0:
    print("Dict is Empty")
print(response)
print(type(response))
print(response[metric])
print(type(response[metric]))
#['list']
#SELECT last(<field name>), time FROM network WHERE device_id = 'x'
#SELECT last(<field name>), time FROM usage WHERE device_id = 'x'

In [ ]:
timestamp_last_influx_datapoint = response[metric].index.time#.to_datetime
print(timestamp_last_influx_datapoint)
print(type(timestamp_last_influx_datapoint))

In [ ]:

print(type(timestamp_last_influx_datapoint))#.strftime("%Y-%m-%dT%H:%M:%SZ")
#timestamp_last_influx_datapoint = datetime.datetime(timestamp_last_influx_datapoint)
#timestamp_last_influx_datapoint.astype("datetime64[ns]")
print(timestamp_last_influx_datapoint)
print(type(timestamp_last_influx_datapoint))

now = datetime.utcnow()
print(now)
print(type(now))
# get timestamp of last datapoint in influxd for each query
#timestamp_last_influx_datapoint = now #- timedelta(hours=settings['timerange'])
# compare with timerange for query and adjust if overlap

starttime = (now - timedelta(hours=settings['timerange']))
if (timestamp_last_influx_datapoint < starttime):
    print('wert in influxDB kleiner als timerange')
    starttime = timestamp_last_influx_datapoint
print(starttime)

## get data from API with dynamic time ranges

In [ ]:
## Request and header parameters
url = 'https://api.gridradar.net/query'
token = config.APItoken#'<token>'

headers = {
    'Content-type': 'application/json', 
    'Authorization': 'Bearer '+token
}


In [ ]:
def convert_dataset_test(metric,response_dict,location):
    outDB=[]
    logging.info("gridradar-API - response: %s", response_dict)
    messdaten=response_dict[0]
        
    for values in messdaten['datapoints']:
        #print(type(values[0]))
        try:
            if math.isnan(values[0])==False:
                outDB_new = {'measurement': metric,
                             'tags': {'location': location},
                             'fields': {'value': values[0]},
                             #'fields': {messdaten['target']: values[0]},
                             'time':timestamp_convert(values[1])}
                outDB.append(outDB_new)
        except Exception as e:
            logging.error(str(e),str(values))
    return(outDB)

In [ ]:
#InfluxDB configuration
import influxdb

influxdb_client = influxdb.InfluxDBClient(
            influx_host,
            influx_port,
            influx_username,
            influx_password,
            influx_database, 
            ssl = False,
            verify_ssl = True
        )

In [ ]:
#dict of available metrics
metrics_dict={
    #"frequency":{"aggr":"100ms", "timerange":1, 'metric_id':1},        #'"1h"'
    #"frequency-mean-1m":{"aggr":"1m", "timerange":720, 'metric_id':2}, #'"30d"'
    #"frequency-mean-1s":{"aggr":"1s", "timerange":12, 'metric_id':3},  #'"12h"'
    #"frequency-ucte-median-1s":{"aggr":"1s", "timerange":24, 'metric_id':4}, #'"1d"'
    #"net-time":{"aggr":"1s", "timerange":24, 'metric_id':5},           #'"1d"'
    "phase-angle":{"aggr":"100ms", "timerange":1, 'metric_id':6},      #'"1h"'
    "phase-angle-first-1m":{"aggr":"1m", "timerange":720, 'metric_id':7}, #'"30d"'
    "phase-angle-first-1m":{"aggr":"1s", "timerange":12, 'metric_id':8} #'"12h"'
}
#metric = "frequency"
#settings = metrics_dict[metric]
#aggr = settings['aggr']
#timerange = settings['timerange']


In [ ]:
#metrics_dict={
#    "frequency":{"aggr":"100ms", "timerange":1, 'metric_id':1}        #'"1h"'
#    #"frequency-mean-1m":{"aggr":"1m", "timerange":720, 'metric_id':2}
#    }
for metric, settings in metrics_dict.items():
    
    ## define API query parameters
    response = getdatafromapi(url,token,metric_request_creator(metric,metrics_dict,location))
    ## Converting the JSON response string to a Python dictionary
    response_dict = json.loads(response.content)
    ## Pretty print response dictionary
    #pprint.pprint(response_dict)
    out_influxDB = convert_dataset_test(metric,response_dict,location)
    #pprint.pprint(out_influxDB)
    influxdb_client.write_points(out_influxDB,database= influx_database,
                                    time_precision="ms",
                                    batch_size=10000,
                                    protocol='json')

## write data to influxdb

In [ ]:
import logging
import math
def convert_dataset_old(response,metric,location,ctr,uptime):
    pprint.pprint(response.status_code)
    logging.debug("API response status code: {}".format(response.status_code))
    #pprint.pprint(response.content)
    ## Converting the JSON response string to a Python dictionary
    result_dict = json.loads(response.content)
    #pprint.pprint(result_dict)
    messdaten=result_dict[0]

    outDB=[]
    for values in messdaten['datapoints']:
        #print(type(values[0]))
        if math.isnan(values[0])==False:
            outDB_new = {
                'measurement': metric,
                'tags': {'location': location},
                'fields': {'value': values[0]},
                'time':timestamp_convert(values[1])
                }
            outDB.append(outDB_new)
    # add response state and time to data output for debugging purposes
    #if metric=='median_frequency':
    #    request_type=1
    #elif metric=='net_time':
    #    request_type=2
    #else:
    #    request_type=0
    #metric_id
    
    debug_info = {
        'measurement': 'API_stats',
        'tags': {'location': location, 'metric':metric},
        'fields': {
            'request_counter': ctr,
            #'request_type':request_type,
            'stat_code':response.status_code,
            'uptime':int(uptime)},
            'time':time_now()}
    outDB.append(debug_info)
    #print(type(outDB))
    #outDB
    return(outDB)

def data_wrangler(url,request,metric,location,headers,ctr,uptime):
    try:
        ctr=ctr+1
        ## Converting the Python dictionary to a JSON string
        json_request = json.dumps(request)
        ## Request execution and response reception
        response = requests.post(url, data=json_request, headers=headers)

        out_influxDB=convert_dataset(response,metric,location,ctr,uptime)
        #pprint.pprint(out_influxDB)
        #logging.debug("Read initial value: {:.2f}".format(m3abs))
        
        client.write_points(out_influxDB, database='gridradar', time_precision='ms', batch_size=10000, protocol='json') 
        #print('gridradar2influx.py  --  request counter: ',str(ctr),'for ',measurement,' written to InfluxDB')
        logging.info('gridradar2influx.py  --  request counter: {} for {} written to InfluxDB'.format(ctr,metric))
        #logging.info("Read initial value: {:.2f}".format(m3abs))
    except (ValueError, TypeError) as ex:
        #debug_str='"%s" cannot be converted to an float: %s' % (raw_content, ex)
        print(ex)
        
    return ctr

In [ ]:
uptime=0
ctr=0
print(metric)
out_influxDB=convert_dataset(response,metric,querypmu,ctr,uptime)

In [ ]:

influxdb_client.write_points(out_influxDB,database= influx_database,
                                    time_precision="ms",
                                    batch_size=10000,
                                    protocol='json')

In [ ]:
dffreq=clientDF.query('SELECT "value" FROM "gridradar"."autogen"."frequency"')
dffreq=dffreq['frequency']#[0]
dffreq=dffreq.rename(columns={'value': 'frequency'})
dffreq['datetime']=dffreq.index
dffreq=dffreq[dffreq["datetime"]>'2023-04-01']
dffreq.head(2)



In [ ]:
dftime=clientDF.query('SELECT "value" FROM "gridradar"."autogen"."net_time"')
dftime=dftime['net_time']
dftime=dftime.rename(columns={'value': 'net_time'})

dftime['datetime']=dftime.index
print(dftime["datetime"].max() - dftime["datetime"].min())
dftime.head(2)

In [ ]:
import plotly.express as px
dfplottime=dftime[dftime["datetime"]>'2022-02-02']
fig = px.line(dfplottime,x="datetime",y="net_time", title='gridradar net_time data from influxDB')
fig.show()